# Data overview for Final Project

## Downloading the data from Statistikbank.dk

The private ownership of cars in Copenhagen has been counted since 2006. The dataset can be found at [Købanhavns Kommunes Statistikbank](kk.statistikbank.dk) under the _Privatbiler_ tab. There are multiple options for which data to extract, based on fuel type, district, family structure, vehicle type etc.. 

For this project, the dataset collected can be found folliwng these steps: 

1. Go to the website and follow these tabs:
- `Privatbiler` 
    - `Personbiler` 
        - `KKBIL1: Privatbiler efter distrikt og drivmiddel`

2. Select all rows in each dropdown, except the "_København i alt_" under _DISTRIKT_ and "_Drivmiddel i alt_" under _DRIVMIDDEL_. 
3. Click "_VIS TABEL_". 
4. Make sure to toggle off "_Inkl. koder i sep. kolonner_" and "_Inkl. fodnoter mv._".
5. Select "_Semikolonsepareret (*.csv)_" in the dropdown "_Åbn / gem som..._".
6. Rename the .csv to "KKBIL.csv"

## Preamble

In [102]:
import folium
from folium.plugins import HeatMap, HeatMapWithTime
import json
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import os

In [103]:
# Set pandas display options to show all columns for .head command
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", None)        # Auto-detect the display width
pd.set_option("display.max_colwidth", None) # Show full content of each column

In [104]:
# Loading the data and files
data_path = os.path.abspath(os.path.join(os.pardir, "projectData"))

# Read shapefile from the provided geojson file
geojson_name = "bydel.geojson"
geojson_path = os.path.join(data_path, geojson_name)

# Load csv data into pandas dataframe
cars_registered_name = "KKBIL.csv"
cars_registered_path = os.path.join(data_path, cars_registered_name)
df = pd.read_csv(cars_registered_path, encoding="ISO-8859-1")#, header=None, skiprows=1)

In [105]:
df.head()

,Benzin,Bydel - Indre By,8675,9015,9016,8831,8758,8572,8489,8429,8549,8671,8921,9003,8991,9122,9276,9593,9250,8810,8493
0,Benzin,Bydel - Østerbro,12302,12666,12870,12782,12592,12354,12184,12022,12162,12337,12631,12901,13238,13344,13530,14098,14048,13470,12842
1,Benzin,Bydel - Nørrebro,8930,9197,9276,8869,8616,8382,8166,8061,8244,8589,8783,9112,9558,9677,9825,10488,10614,10157,9720
2,Benzin,Bydel - Vesterbro/Kongens Enghave,6634,7248,7499,7733,7753,7846,7636,7600,7868,8105,8381,8767,9200,9606,9949,10616,10987,10611,10045
3,Benzin,Bydel - Valby,8867,9094,9166,9114,9121,9102,9002,9020,9228,9472,9643,9888,10234,10541,10810,11477,11510,11237,10821
4,Benzin,Bydel - Vanløse,8146,8289,8262,8097,8033,8006,7945,7937,7952,8110,8296,8313,8451,8548,8625,8817,8843,8500,8291


In [108]:
# Manually assign the start and end year of the dataset, depending on time fetched from website
start_y = 2006
end_y = 2024

new_cols = ["CarFuelType", "CityDistrict", *list(range(start_y, end_y+1, 1))]

# Shift table downwards, so that current header becomes first row, then assign the new custom header
df_ncols = (df.T.reset_index().T.reset_index(drop=True).set_axis(new_cols, axis=1))

In [111]:
df_ncols.head()

,CarFuelType,CityDistrict,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Benzin,Bydel - Indre By,8675,9015,9016,8831,8758,8572,8489,8429,8549,8671,8921,9003,8991,9122,9276,9593,9250,8810,8493
1,Benzin,Bydel - Østerbro,12302,12666,12870,12782,12592,12354,12184,12022,12162,12337,12631,12901,13238,13344,13530,14098,14048,13470,12842
2,Benzin,Bydel - Nørrebro,8930,9197,9276,8869,8616,8382,8166,8061,8244,8589,8783,9112,9558,9677,9825,10488,10614,10157,9720
3,Benzin,Bydel - Vesterbro/Kongens Enghave,6634,7248,7499,7733,7753,7846,7636,7600,7868,8105,8381,8767,9200,9606,9949,10616,10987,10611,10045
4,Benzin,Bydel - Valby,8867,9094,9166,9114,9121,9102,9002,9020,9228,9472,9643,9888,10234,10541,10810,11477,11510,11237,10821
